# Simulus Simple Use Cases

Simulus is a discrete-event simulator in Python. This document is to demonstrate how to run simulus via a few examples. This is not a tutorial. For that, try [Simulus Basics](simulus-basics.ipynb). All the examples shown in this guide can be found under the `examples/simple-tests` directory in the simulus source-code distribution.

Simulus uses [greenlet](https://pypi.org/project/greenlet/) to implement the process-oriented simulation world-view. Because of that, you need install greenlet using something like the following (you may need to run it in sudo):

In [1]:
%%script bash

pip install greenlet

You'll also need to set PYTHONPATH to include the source directory. If you are running this jupyter notebook from the doc directory, you should find its parent directory in the PYTHONPATH:

In [2]:
%env PYTHONPATH

'/Users/liux/Workspace/projects/gitlab/simulus:.'

If all are fine at this point, you can simply import the module 'simulus' to start using the simulator. 

In [3]:
import simulus

### Use Case #1: Direct Event Scheduling

One can schedule functions to be executed at designated simulation time. The functions in this case are called event handlers (using the discrete-event simulation terminology).

In [4]:
# %load "../examples/simple-case1.py"
import simulus

# An event handler for simulus is a python function with two parameters:
# the first parameter is the simulator, and the second parameters is
# the dictionary containing user-defined parameters for the event handler
def myfunc(sim, params):
    print(str(sim.now) + ": myfunc() runs with params=" + str(params))
    # schedule the next event at 5 seconds from now
    sim.sched(myfunc, params=params, offset=5)

# create an anonymous simulator
sim1 = simulus.simulator() 

# schedule the first event at 10 seconds
sim1.sched(myfunc, until=10, msg="hello world", value=100)

# advance simulation until 100 seconds
sim1.run(until=100)
print("simulator.run() ends at " + str(sim1.now))

# we can advance simulation for another 50 seconds
sim1.run(offset=50)
print("simulator.run() ends at " + str(sim1.now))


10: myfunc() runs with params={'msg': 'hello world', 'value': 100}
15: myfunc() runs with params={'msg': 'hello world', 'value': 100}
20: myfunc() runs with params={'msg': 'hello world', 'value': 100}
25: myfunc() runs with params={'msg': 'hello world', 'value': 100}
30: myfunc() runs with params={'msg': 'hello world', 'value': 100}
35: myfunc() runs with params={'msg': 'hello world', 'value': 100}
40: myfunc() runs with params={'msg': 'hello world', 'value': 100}
45: myfunc() runs with params={'msg': 'hello world', 'value': 100}
50: myfunc() runs with params={'msg': 'hello world', 'value': 100}
55: myfunc() runs with params={'msg': 'hello world', 'value': 100}
60: myfunc() runs with params={'msg': 'hello world', 'value': 100}
65: myfunc() runs with params={'msg': 'hello world', 'value': 100}
70: myfunc() runs with params={'msg': 'hello world', 'value': 100}
75: myfunc() runs with params={'msg': 'hello world', 'value': 100}
80: myfunc() runs with params={'msg': 'hello world', 'value': 

### Use Case #2: Simulation Process

A simulation process is an independent thread of execution. A process can be blocked and therefore advances its simulation time either by sleeping for some duration of time or by being blocked from synchronization primitives (such as semaphores).

In [5]:
# %load "../examples/simple-case2.py"
import simulus

# A process for simulus is a python function with two parameters: 
# the first parameter is the simulator, and the second parameter is
# the dictionary containing user-defined parameters for the process
def myproc(sim, params):
    intv = params.get("intv")
    id = params.get("id")
    print(str(sim.now) + ": myproc(%d) runs with params=%r" % (id, params))
    while True:
        # suspend the process for some time
        sim.sleep(intv)
        print(str(sim.now) + ": myproc(%d) resumes execution" % id)

# create an anonymous simulator
sim2 = simulus.simulator()

# start a process 100 seconds from now
sim2.process(myproc, offset=100, intv=10, id=0)
# start another process 5 seconds from now
sim2.process(myproc, offset=5, intv=20, id=1)

# advance simulation until 200 seconds
sim2.run(until=200)
print("simulator.run() ends at " + str(sim2.now))

sim2.run(offset=50)
print("simulator.run() ends at " + str(sim2.now))


5: myproc(1) runs with params={'intv': 20, 'id': 1}
25: myproc(1) resumes execution
45: myproc(1) resumes execution
65: myproc(1) resumes execution
85: myproc(1) resumes execution
100: myproc(0) runs with params={'intv': 10, 'id': 0}
105: myproc(1) resumes execution
110: myproc(0) resumes execution
120: myproc(0) resumes execution
125: myproc(1) resumes execution
130: myproc(0) resumes execution
140: myproc(0) resumes execution
145: myproc(1) resumes execution
150: myproc(0) resumes execution
160: myproc(0) resumes execution
165: myproc(1) resumes execution
170: myproc(0) resumes execution
180: myproc(0) resumes execution
185: myproc(1) resumes execution
190: myproc(0) resumes execution
200: myproc(0) resumes execution
simulator.run() ends at 200
205: myproc(1) resumes execution
210: myproc(0) resumes execution
220: myproc(0) resumes execution
225: myproc(1) resumes execution
230: myproc(0) resumes execution
240: myproc(0) resumes execution
245: myproc(1) resumes execution
250: myproc(

### Use Case #3: Process Synchronization with Semaphores

We illustrate the use of semaphore in the context of a classic producer-consumer problem. We are simulating a single-server queue (M/M/1) here.

In [6]:
# %load "../examples/simple-case3.py"
import simulus

from random import seed, expovariate
from statistics import mean, median, stdev

# make it repeatable
seed(12345) 

# configuration of the single server queue: the mean inter-arrival
# time, and the mean service time
cfg = {"mean_iat":1, "mean_svc":0.8}

# keep the time of job arrivals, starting services, and departures
arrivals = []
starts = []
finishes = []

# the producer process waits for some random time from an 
# exponential distribution, and increments the semaphore 
# to represent a new item being produced, and then repeats 
def producer(sim, params):
    mean_iat = params.get("mean_iat")
    sem = params.get("sem")
    while True:
        iat = expovariate(1.0/mean_iat)
        sim.sleep(iat)
        print("%g: job arrives (iat=%g)" % (sim.now, iat))
        arrivals.append(sim.now)
        sem.signal()
        
# the consumer process waits for the semaphore (it decrements
# the value and blocks if the value is non-positive), waits for
# some random time from another exponential distribution, and
# then repeats
def consumer(sim, params):
    mean_svc = params.get("mean_svc")
    sem = params.get("sem")
    while True:
        sem.wait()
        print("%g: job starts service" % sim.now)
        starts.append(sim.now)
        svc = expovariate(1.0/mean_svc)
        sim.sleep(svc)
        print("%g: job departs (svc=%g)" % (sim.now, svc))
        finishes.append(sim.now)

# create an anonymous simulator
sim3 = simulus.simulator()

# create a semaphore with initial value of zero
sem = sim3.semaphore(0)

# start the producer and consumer processes
sim3.process(producer, params=cfg, sem=sem)
sim3.process(consumer, params=cfg, sem=sem)

# advance simulation until 100 seconds
sim3.run(until=100)
print("simulator.run() ends at " + str(sim3.now))

# calculate and output statistics
print(f'Results: jobs=arrival:{len(arrivals)}, starts:{len(starts)}, finishes:{len(finishes)}')
waits = [start - arrival for arrival, start in zip(arrivals, starts)]
totals = [finish - arrival for arrival, finish in zip(arrivals, finishes)]
print(f'Wait Time: mean={mean(waits):.1f}, stdev={stdev(waits):.1f}, median={median(waits):.1f}.  max={max(waits):.1f}')
print(f'Total Time: mean={mean(totals):.1f}, stdev={stdev(totals):.1f}, median={median(totals):.1f}.  max={max(totals):.1f}')
my_lambda = 1.0/cfg['mean_iat'] # mean arrival rate
my_mu = 1.0/cfg['mean_svc'] # mean service rate
my_rho = my_lambda/my_mu # server utilization
my_lq = my_rho*my_rho/(1-my_rho) # number in queue
my_wq = my_lq/my_lambda # wait in queue
my_w = my_wq+1/my_mu # wait in system
print(f'Theoretical Results: mean wait time = {my_wq:.1f}, mean total time = {my_w:.1f}')


0.538916: job arrives (iat=0.538916)
0.538916: job starts service
0.549138: job arrives (iat=0.0102212)
0.903871: job arrives (iat=0.354734)
1.36339: job arrives (iat=0.459518)
1.57864: job arrives (iat=0.215251)
1.93424: job departs (svc=1.39532)
1.93424: job starts service
2.07533: job departs (svc=0.141092)
2.07533: job starts service
2.18148: job departs (svc=0.106155)
2.18148: job starts service
2.41337: job arrives (iat=0.83473)
2.63531: job departs (svc=0.453827)
2.63531: job starts service
2.78857: job departs (svc=0.153261)
2.78857: job starts service
3.23909: job arrives (iat=0.825716)
3.43312: job departs (svc=0.644553)
3.43312: job starts service
3.67744: job arrives (iat=0.438352)
3.77317: job arrives (iat=0.0957338)
5.97043: job departs (svc=2.5373)
5.97043: job starts service
6.39541: job departs (svc=0.424985)
6.39541: job starts service
6.95625: job departs (svc=0.560839)
7.61941: job arrives (iat=3.84624)
7.61941: job starts service
7.77975: job arrives (iat=0.16034)


### Use Case #4: Dynamic Processes

We continue with the previous example. At the time, rathar than using semaphores, we can achieve (exactly) the same results using simulus functions for dynamically creating processes.

In [7]:
# %load "../examples/simple-case4.py"
import simulus

from random import seed, expovariate
from statistics import mean, median, stdev

# make it repeatable
seed(12345) 

# configuration of the single server queue: the mean inter-arrival
# time, and the mean service time
cfg = {"mean_iat":1, "mean_svc":0.8}

# keep the time of job arrivals, starting services, and departures
arrivals = []
starts = []
finishes = []

# we keep the account of the number of jobs in the system (those who
# have arrived but not yet departed); this is used to indicate whether
# there's a consumer process currently running; the value is more than
# 1, we don't need to create a new consumer process
jobs_in_system = 0

# the producer process waits for some random time from an exponential
# distribution to represent a new item being produced, creates a
# consumer process when necessary to represent the item being
# consumed, and then repeats
def producer(sim, params):
    global jobs_in_system
    mean_iat = cfg["mean_iat"]
    while True:
        iat = expovariate(1.0/mean_iat)
        sim.sleep(iat)
        print("%g: job arrives (iat=%g)" % (sim.now, iat))
        arrivals.append(sim.now)
        jobs_in_system += 1
        if jobs_in_system <= 1:
            sim.process(consumer)
        
# the consumer process waits for the semaphore (it decrements
# the value and blocks if the value is non-positive), waits for
# some random time from another exponential distribution, and
# then repeats
def consumer(sim, params):
    global jobs_in_system
    mean_svc = cfg["mean_svc"]
    while jobs_in_system > 0:
        print("%g: job starts service" % sim.now)
        starts.append(sim.now)
        svc = expovariate(1.0/mean_svc)
        sim.sleep(svc)
        print("%g: job departs (svc=%g)" % (sim.now, svc))
        finishes.append(sim.now)
        jobs_in_system -= 1

# create an anonymous simulator
sim3 = simulus.simulator()

# start the producer process only
sim3.process(producer)

# advance simulation until 100 seconds
sim3.run(until=100)
print("simulator.run() ends at " + str(sim3.now))

# calculate and output statistics
print(f'Results: jobs=arrival:{len(arrivals)}, starts:{len(starts)}, finishes:{len(finishes)}')
waits = [start - arrival for arrival, start in zip(arrivals, starts)]
totals = [finish - arrival for arrival, finish in zip(arrivals, finishes)]
print(f'Wait Time: mean={mean(waits):.1f}, stdev={stdev(waits):.1f}, median={median(waits):.1f}.  max={max(waits):.1f}')
print(f'Total Time: mean={mean(totals):.1f}, stdev={stdev(totals):.1f}, median={median(totals):.1f}.  max={max(totals):.1f}')
my_lambda = 1.0/cfg['mean_iat'] # mean arrival rate
my_mu = 1.0/cfg['mean_svc'] # mean service rate
my_rho = my_lambda/my_mu # server utilization
my_lq = my_rho*my_rho/(1-my_rho) # number in queue
my_wq = my_lq/my_lambda # wait in queue
my_w = my_wq+1/my_mu # wait in system
print(f'Theoretical Results: mean wait time = {my_wq:.1f}, mean total time = {my_w:.1f}')


0.538916: job arrives (iat=0.538916)
0.538916: job starts service
0.549138: job arrives (iat=0.0102212)
0.903871: job arrives (iat=0.354734)
1.36339: job arrives (iat=0.459518)
1.57864: job arrives (iat=0.215251)
1.93424: job departs (svc=1.39532)
1.93424: job starts service
2.07533: job departs (svc=0.141092)
2.07533: job starts service
2.18148: job departs (svc=0.106155)
2.18148: job starts service
2.41337: job arrives (iat=0.83473)
2.63531: job departs (svc=0.453827)
2.63531: job starts service
2.78857: job departs (svc=0.153261)
2.78857: job starts service
3.23909: job arrives (iat=0.825716)
3.43312: job departs (svc=0.644553)
3.43312: job starts service
3.67744: job arrives (iat=0.438352)
3.77317: job arrives (iat=0.0957338)
5.97043: job departs (svc=2.5373)
5.97043: job starts service
6.39541: job departs (svc=0.424985)
6.39541: job starts service
6.95625: job departs (svc=0.560839)
7.61941: job arrives (iat=3.84624)
7.61941: job starts service
7.77975: job arrives (iat=0.16034)
